In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import numpy as np

from sklearn.preprocessing import scale
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import os

from geomloss import SamplesLoss

from imputers import OTimputer, RRimputer

from data_scaler import DataScaler

from utils import *
from data_loaders import dataset_loader
from softimpute import softimpute, cv_softimpute

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")

torch.set_default_tensor_type('torch.DoubleTensor')

In [ ]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 读取 CSV 文件
df = pd.read_csv("juhe.csv")
df

## scaler and thinking of the miss data

In [ ]:
from data_scaler import DataScaler
import numpy as np
import pandas as pd

# 计算每列的均值和标准差时，忽略缺失值
data = df.to_numpy().astype(float)
#df_data = pd.DataFrame(data)  # 将numpy数组转换为DataFrame

scaler = DataScaler()
mask = np.isnan(data)
complete_data = data[~np.any(mask, axis=1)]  # 选择没有缺失值的行
# 修改这行代码
scaler.fit(complete_data)  # 先fit
Mean, Std = scaler.mean_, scaler.std_  # 然后获取均值和标准差
# 然后对整个数据集进行转换
data_for_im_np = scaler.transform(data)






# 然后对整个数据集进行转换
data_for_im_np = scaler.transform(data)



data_for_im = torch.from_numpy(data_for_im_np)
#X_true = torch.from_numpy(ground_truth)

In [ ]:
n, d = data_for_im.shape
batchsize = 128 # If the batch size is larger than half the dataset's size,
                # it will be redefined in the imputation methods.
lr = 1e-2
epsilon = pick_epsilon(data_for_im) #

In [ ]:
sk_imputer = OTimputer(eps=epsilon, batchsize=batchsize, lr=lr, niter=2000)

In [ ]:
sk_imp = sk_imputer.fit_transform(data_for_im, verbose=True, report_interval=500)

## Coming Back

In [ ]:
imp = sk_imp.detach().cpu().numpy()
#df_imp =scaler.inverse_transform(imp)
df_imp = pd.DataFrame(imp)   # 先detach再转换为numpy数组

# 保存为CSV文件
df_imp.to_csv('imputed_data2.csv', index=False)